## MAKE SURE YOU UPDATED your ssh key

In [1]:
import os
import subprocess
from datetime import datetime

In [2]:
cscs_username = 'cpi' # replace 'someuser' with your CSCS 'username'
cscs_account = 's1267'
new_host_label = 'daint.alps'
remotehost = "daint.alps.cscs.ch"  # Ensure this is the correct hostname

In [3]:
alps_files = "/home/jovyan/aiidalab-alps-files"
config_path = f"/home/jovyan/.ssh/config"
config_source = f"/home/jovyan/aiidalab-alps-files/config"
config_without_ela = f"/home/jovian/aiidalab-alps-files/config_without_ela"

In [4]:
#repo_url="https://github.com/nanotech-empa/aiidalab-alps-files.git"  # git repository with files for ALPS
mydir = "/home/jovyan/"  # Directory containing the repo
repo_name = "aiidalab-alps-files"
repo_path = os.path.join(mydir, repo_name)

#if not os.path.isdir(repo_path):  # Check if the directory exists
#    print(f"Cloning {repo_url} into {repo_path}...")
#    subprocess.run(f"cd {mydir} && git clone {repo_url}", shell=True, check=True)
#else:
#    print(f"Repository {repo_name} exists, pulling latest changes...")
#    subprocess.run(f"cd {repo_path} && git pull", shell=True, check=True)

In [5]:
# update the config file in .ssh and set teh correct user in the .yml files
relabeled_host = datetime.now().strftime("Used_till_%Y%m%d%H%M")+'_'+new_host_label

# set .ssh/config file
config_command = f"""
if grep -q 'daint.alps.cscs.ch' {config_path} 2>/dev/null; then
    echo 'No changes needed for .ssh/config: daint.alps.cscs.ch already present in {config_path}';
else
    if [ ! -f {config_path} ]; then
        cp {config_source} {config_path};
    else
        if grep -q 'ela' {config_path}; then
            cat {config_without_ela} {config_path} > {config_path}.tmp && mv {config_path}.tmp {config_path};
        else
            cat {config_source} {config_path} > {config_path}.tmp && mv {config_path}.tmp {config_path};
        fi
    fi
    sed -i 's/cscsusername/{cscs_username}/g' {config_path}
    echo 'Updated {config_path} successfully!';
fi
sed -i 's/cscsuser/{cscs_username}/g' {alps_files}/alps_setup.yml
sed -i 's/cscsaccount/{cscs_account}/g' {alps_files}/alps_setup.yml
sed -i 's/cscsuser/{cscs_username}/g' {alps_files}/alps_config.yml
sed -i 's/cscsuser/{cscs_username}/g' {alps_files}/cp2k.yml
"""
# relabel old computers
relabel_command = f"""if verdi computer list  | grep -q '{new_host_label}'; then 
    verdi computer relabel {new_host_label} {relabeled_host} && 
    verdi computer disable {relabeled_host} aiida@localhost; 
fi"""

# Hide old codes that will be already @relabeled_host 
hide_code_command = f"verdi code list | awk 'NR>2 {{print $1}}' | grep '@{relabeled_host}' | xargs -r -I {{}} verdicode hide {{}}"

# setup ALPS
alps_setup_command = f"verdi computer setup --config alps_setup.yml"
alps_config_command = f"verdi computer configure core.ssh daint.alps --config alps_config.yml"

## set .ssh/config file, relable old daint.alps, hide old codes_at_daint.alps

In [6]:
local_commands=[config_command,relabel_command,hide_code_command,alps_setup_command,alps_config_command]
for command in local_commands:
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    # Print the output
    print(result.stdout)
    print(result.stderr)

No changes needed for .ssh/config: daint.alps.cscs.ch already present in /home/jovyan/.ssh/config






Success: Computer<38> daint.alps created
Report: Note: before the computer can be used, it has to be configured with the command:
Report:   verdi -p default computer configure core.ssh daint.alps


Report: Configuring computer daint.alps for user aiida@localhost.
Success: daint.alps successfully configured for aiida@localhost




## do first ssh connection

In [7]:
ela = "ela.cscs.ch"
daint = "daint.alps.cscs.ch"

# check if ssh works
scan_keys = False
ssh_works = False
result = subprocess.run(f"ssh {daint} 'ls /users/{cscs_username}' ", shell=True, capture_output=True, text=True)
if result.returncode !=0:
    scan_keys = True
else:
    ssh_works = True

# Loop over each host and add it to known_hosts
if scan_keys:
    print(f"Adding {ela} to known_hosts...")
    ssh_keyscan_command = f"ssh-keyscan -H {ela} >> ~/.ssh/known_hosts"
    result = subprocess.run(ssh_keyscan_command, shell=True, capture_output=True, text=True)
    # Print output for debugging
    print("STDOUT:", result.stdout)
    print("STDERR:", result.stderr)

    print(f"Adding {daint} to known_hosts...")
    ssh_keyscan_command = f"ssh {ela} ssh-keyscan -H {daint} >> ~/.ssh/known_hosts"
    result = subprocess.run(ssh_keyscan_command, shell=True, capture_output=True, text=True)
    result = subprocess.run(ssh_keyscan_command, shell=True, capture_output=True, text=True)
    # Print output for debugging
    print("STDOUT:", result.stdout)
    print("STDERR:", result.stderr)

    # check if ssh works
    result = subprocess.run(f"ssh {daint} 'ls /users/{cscs_username}' ", shell=True, capture_output=True, text=True)
    if result.returncode !=0:
        print(f"Ask for help, something in the ssh to {daint} connection is wrong")
    else:
        ssh_works = True

if ssh_works:
    print(f"ssh to {daint} seems to work")


ssh to daint.alps.cscs.ch seems to work


## setup codes

In [8]:
for code in ['cp2k.yml','stm.yml','pw.yml','pp.yml','projwfc.yml','dos.yml']:
    command = f"verdi code create core.code.installed --config {code}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    # Print the output
    print(result.stdout)
    print(result.stderr)
    

Success: Created InstalledCode<14390>


Success: Created InstalledCode<14391>


Success: Created InstalledCode<14392>


Success: Created InstalledCode<14393>


Success: Created InstalledCode<14394>


Success: Created InstalledCode<14395>




## creating directories on daint

In [ ]:
remote_commands = [, f"cd /users/{cscs_username}/src && git clone {repo_url}"]

In [ ]:
for command in remote_commands:
    ssh_command = f"ssh {remotehost} '{command}'"
    result = subprocess.run(ssh_command, shell=True, capture_output=True, text=True)
    # Print the output
    print(result.stdout)
    print(result.stderr)

## copying scripts and data directories

In [ ]:
# Paths on the remote host
remotehost='daint.alps.cscs.ch'

# Define the remote command
uenv_command = "uenv repo create"
pull_cp2k = "uenv image pull cp2k/2024.3:v2"
pull_qe = "uenv image pull quantumespresso/v7.4:v2"
pull_stm = "uenv image pull service::cp2k-spm-tools/1.5.0:1658742098"
make_src = f"mkdir -p /users/{cscs_username}/src"
scp_mps = f"scp {alps_files}/mps-wrapper.sh {remotehost}:/users/{cscs_username}/bin/"
scp_cp2kdatadir = f"scp {alps_files}/cp2k {remotehost}:/users/{cscs_username}/src/"


ssh_commands = [pull_cp2k,pull_qe,pull_stm,make_src]
scp_commands = [scp_mps,scp_cp2kdatadir]


# Construct the SSH command
for command in ssh_commands:
    ssh_command = f"ssh {remotehost} '{command}'"
    result = subprocess.run(ssh_command, shell=True, capture_output=True, text=True)
for command in scp_commands:
    result = subprocess.run(command, shell=True, capture_output=True, text=True)

ssh_commands = []

# Execute the command


# Print the output
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)
